In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
FOLDERNAME='ML_final_project'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings
warnings.filterwarnings('ignore')

from scripts.data_processor import DataPreprocessor

In [ ]:
!pip install numpy==1.26.4 --force-reinstall -q
!pip install mlflow==2.15.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84

In [ ]:
!pip install dagshub mlflow -q

import dagshub
import mlflow
import mlflow.lightgbm
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException
from mlflow.entities import Experiment
from mlflow.entities.lifecycle_stage import LifecycleStage

dagshub.init(repo_owner='skara-21', repo_name='ML_Final_Project', mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/skara-21/ML_Final_Project.mlflow')

client = MlflowClient()
experiment_name = "LightGBM_Training"

try:
    experiment = client.get_experiment_by_name(experiment_name)
    if experiment:
        # If it exists and is deleted, restore it or permanently delete it
        if experiment.lifecycle_stage == LifecycleStage.DELETED:
            print(f"Experiment '{experiment_name}' is deleted. Attempting to restore...")
            try:
                client.restore_experiment(experiment.experiment_id)
                print(f"Experiment '{experiment_name}' restored.")
            except Exception as e:
                print(f"Failed to restore experiment: {e}. Permanently deleting...")
                client.delete_experiment(experiment.experiment_id, True)
                print(f"Experiment '{experiment_name}' permanently deleted.")
                # Now create the experiment again
                experiment = None # Set experiment to None to trigger creation


    if experiment is None:
         # If experiment does not exist or was permanently deleted, create it
        experiment_id = client.create_experiment(experiment_name)
        print(f"Experiment '{experiment_name}' created with ID: {experiment_id}")
    else:
        # If experiment exists and is active, set it
        mlflow.set_experiment(experiment_name)
        experiment_id = experiment.experiment_id
        print(f"Experiment '{experiment_name}' set with ID: {experiment_id}")

except MlflowException as e:
    print(f"An MLflow error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.6 MB/s eta 0:00:00


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d2f01277-7e9d-496f-961f-dfe8c8d06cbf&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=9ac2b6d499d23f900ed3faf81dccded798de4b21303235add62ec9ccc76d48e7




Output()

Accessing as skara-21

Initialized MLflow to track repo "skara-21/ML_Final_Project"

Repository skara-21/ML_Final_Project initialized!

Experiment 'LightGBM_Training' set with ID: 1


In [ ]:
print(f"MLflow experiment set to: LightGBM_Training")
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")

MLflow experiment set to: LightGBM_Training
MLflow tracking URI: https://dagshub.com/skara-21/ML_Final_Project.mlflow


In [ ]:
def wmae_score(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

def check_missing(df, dataset_name):
    """Check missing values in dataframe"""
    missing_data = {}
    for col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            missing_data[col] = missing_count
            print(f"{dataset_name} - {col}: {missing_count} missing values")
    return missing_data

In [ ]:
with mlflow.start_run(run_name="LightGBM_Data_Loading") as run:
    FOLDERNAME='/content/drive/MyDrive/ML_final_project'
    DATAPATH=f'{FOLDERNAME}/data/'

    train_df=pd.read_csv(DATAPATH+'train.csv')
    test_df=pd.read_csv(DATAPATH+'test.csv')
    features_df=pd.read_csv(DATAPATH+'features.csv')
    stores_df=pd.read_csv(DATAPATH+'stores.csv')

    mlflow.log_metric("initial_train_rows", train_df.shape[0])
    mlflow.log_metric("initial_train_cols", train_df.shape[1])
    mlflow.log_metric("initial_test_rows", test_df.shape[0])
    mlflow.log_metric("initial_test_cols", test_df.shape[1])
    mlflow.log_metric("features_rows", features_df.shape[0])
    mlflow.log_metric("stores_rows", stores_df.shape[0])

    print(f"Data loaded:")
    print(f"Train: {train_df.shape}")
    print(f"Test: {test_df.shape}")
    print(f"Features: {features_df.shape}")
    print(f"Stores: {stores_df.shape}")

mlflow.end_run()

2025/07/15 11:08:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Data_Loading at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/081e62ae1bd942b5ad9dc194d86d0887.
2025/07/15 11:08:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.


Data loaded:
Train: (421570, 5)
Test: (115064, 4)
Features: (8190, 12)
Stores: (45, 3)


In [ ]:
with mlflow.start_run(run_name="LightGBM_Cleaning") as run:
    features_df['Date']=pd.to_datetime(features_df['Date'])
    train_df['Date']=pd.to_datetime(train_df['Date'])
    test_df['Date']=pd.to_datetime(test_df['Date'])

    initial_rows=len(train_df)
    train_df=train_df[train_df['Weekly_Sales']>=0].reset_index(drop=True)
    removed_ones = initial_rows - len(train_df)

    mlflow.log_metric("negative_sales_removed", removed_ones)
    mlflow.log_metric("cleaned_train_rows", len(train_df))

    print(f"Removed {removed_ones} rows with negative sales")
    print(f"Final training rows: {len(train_df)}")

mlflow.end_run()

2025/07/15 11:08:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Cleaning at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/b8dae086d2814233a0188556e90aeef5.
2025/07/15 11:08:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.


Removed 1285 rows with negative sales
Final training rows: 420285


In [ ]:
with mlflow.start_run(run_name="LightGBM_Missing_Values") as run:
    train_merged = train_df.merge(stores_df, on='Store', how='left')
    train_merged = train_merged.merge(features_df, on=['Store', 'Date'], how='left')

    test_merged = test_df.merge(stores_df, on='Store', how='left')
    test_merged = test_merged.merge(features_df, on=['Store', 'Date'], how='left')

    train_missing = check_missing(train_merged, "Train Merged")
    test_missing = check_missing(test_merged, "Test Merged")

    for col, count in train_missing.items():
        mlflow.log_metric(f"train_missing_{col}", count)
    for col, count in test_missing.items():
        mlflow.log_metric(f"test_missing_{col}", count)

    mlflow.log_metric("total_train_missing_columns", len(train_missing))
    mlflow.log_metric("total_test_missing_columns", len(test_missing))

mlflow.end_run()

Train Merged - MarkDown1: 270085 missing values
Train Merged - MarkDown2: 309367 missing values
Train Merged - MarkDown3: 283618 missing values
Train Merged - MarkDown4: 285750 missing values
Train Merged - MarkDown5: 269337 missing values
Test Merged - MarkDown1: 149 missing values
Test Merged - MarkDown2: 28627 missing values
Test Merged - MarkDown3: 9829 missing values
Test Merged - MarkDown4: 12888 missing values
Test Merged - CPI: 38162 missing values
Test Merged - Unemployment: 38162 missing values


2025/07/15 11:09:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Missing_Values at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/fb1892f11aec4081b0b5b65a21f91d23.
2025/07/15 11:09:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.


In [ ]:
def time_series_split_walmart(train_data, n_splits=5):
    train_data=train_data.sort_values('Date')
    dates=train_data['Date'].unique()
    dates=np.sort(dates)

    split_size=len(dates)//(n_splits+1)
    splits=[]

    for i in range(n_splits):
        train_end_idx=(i+1) * split_size
        val_start_idx=train_end_idx
        val_end_idx=train_end_idx+split_size

        train_dates=dates[:train_end_idx]
        val_dates=dates[val_start_idx:val_end_idx]

        train_idx=train_data[train_data['Date'].isin(train_dates)].index
        val_idx=train_data[train_data['Date'].isin(val_dates)].index

        splits.append((train_idx, val_idx))

    return splits

In [ ]:
def run_cross_validation(train_data):
    with mlflow.start_run(run_name="LightGBM_Cross_Validation") as run:
        lgb_params={
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.03,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'max_depth': 8,
            'min_child_samples': 20,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'random_state': 42,
            'n_estimators': 2000
        }

        for param, value in lgb_params.items():
            mlflow.log_param(f"cv_{param}", value)

        mlflow.log_param("cv_splits", 5)
        mlflow.log_param("cv_strategy", "TimeSeriesSplit")

        feature_cols=[col for col in train_data.columns if col != 'Weekly_Sales']
        X=train_data[feature_cols]
        y=train_data['Weekly_Sales']

        weights=train_data['IsHoliday'].map({True: 5, False: 1}).values

        splits=time_series_split_walmart(train_data, n_splits=5)
        cv_scores=[]
        wmae_scores=[]

        for fold, (train_idx, val_idx) in enumerate(splits):
            print(f"Working on fold {fold + 1}/5")

            X_train_fold=X.iloc[train_idx]
            y_train_fold=y.iloc[train_idx]
            X_val_fold=X.iloc[val_idx]
            y_val_fold=y.iloc[val_idx]
            weights_val=weights[val_idx]

            val_data_with_target=train_data.iloc[val_idx]

            preprocessor=DataPreprocessor(
                stores_df=stores_df,
                features_df=features_df,
                lag_features=[1, 2, 4, 8, 12],
                rolling_windows=[4, 8, 12]
            )

            X_train_processed=preprocessor.fit(X_train_fold, y_train_fold).transform(X_train_fold)
            X_val_processed=preprocessor.transform(val_data_with_target)

            if 'Weekly_Sales' in X_val_processed.columns:
                X_val_processed=X_val_processed.drop('Weekly_Sales', axis=1)

            model=lgb.LGBMRegressor(**lgb_params)
            model.fit(X_train_processed, y_train_fold,
                     eval_set=[(X_val_processed, y_val_fold)],
                     callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)])

            y_pred=model.predict(X_val_processed)
            y_pred=np.maximum(y_pred, 0)

            mae=mean_absolute_error(y_val_fold, y_pred)
            rmse=np.sqrt(mean_squared_error(y_val_fold, y_pred))
            r2=r2_score(y_val_fold, y_pred)
            wmae=wmae_score(y_val_fold, y_pred, weights_val)

            cv_scores.append({'fold': fold, 'mae': mae, 'rmse': rmse, 'r2': r2, 'wmae': wmae})
            wmae_scores.append(wmae)

            mlflow.log_metric(f"fold_{fold}_mae", mae)
            mlflow.log_metric(f"fold_{fold}_rmse", rmse)
            mlflow.log_metric(f"fold_{fold}_r2", r2)
            mlflow.log_metric(f"fold_{fold}_wmae", wmae)

            print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}, WMAE: {wmae:.2f}")

        avg_mae=np.mean([score['mae'] for score in cv_scores])
        avg_rmse=np.mean([score['rmse'] for score in cv_scores])
        avg_r2=np.mean([score['r2'] for score in cv_scores])
        avg_wmae=np.mean(wmae_scores)

        mlflow.log_metric("cv_avg_mae", avg_mae)
        mlflow.log_metric("cv_avg_rmse", avg_rmse)
        mlflow.log_metric("cv_avg_r2", avg_r2)
        mlflow.log_metric("cv_avg_wmae", avg_wmae)

        print(f"\nCV results:")
        print(f"Average MAE: {avg_mae:.2f}")
        print(f"Average RMSE: {avg_rmse:.2f}")
        print(f"Average R2: {avg_r2:.4f}")
        print(f"Average WMAE: {avg_wmae:.2f}")

    mlflow.end_run()
    return cv_scores, avg_wmae

In [ ]:
def train_model(train_data):
    with mlflow.start_run(run_name="LightGBM_Final_Training") as run:
        final_params={
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.03,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'max_depth': 8,
            'min_child_samples': 20,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'random_state': 42,
            'n_estimators': 2000
        }

        for param, value in final_params.items():
            mlflow.log_param(param, value)

        mlflow.log_param("lag_features", [1, 2, 4, 8, 12])
        mlflow.log_param("rolling_windows", [4, 8, 12])
        mlflow.log_param("preprocessor", "DataPreprocessor")

        pipeline=Pipeline([
            ('preprocessor', DataPreprocessor(
                stores_df=stores_df,
                features_df=features_df,
                lag_features=[1, 2, 4, 8, 12],
                rolling_windows=[4, 8, 12]
            )),
            ('model', lgb.LGBMRegressor(**final_params))
        ])

        feature_cols=[col for col in train_data.columns if col != 'Weekly_Sales']
        X_train=train_data[feature_cols]
        y_train=train_data['Weekly_Sales']

        pipeline.fit(X_train, y_train)

        train_pred=pipeline.predict(X_train)
        train_pred=np.maximum(train_pred, 0)

        mae=mean_absolute_error(y_train, train_pred)
        rmse=np.sqrt(mean_squared_error(y_train, train_pred))
        r2=r2_score(y_train, train_pred)

        mlflow.log_metric("final_mae", mae)
        mlflow.log_metric("final_rmse", rmse)
        mlflow.log_metric("final_r2", r2)

        lgb_model = pipeline.named_steps['model']
        mlflow.lightgbm.log_model(
            lgb_model,
            "lightgbm_model",
            registered_model_name="walmart_lightgbm_model"
        )


        print(f"Model training metrics:")
        print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}")

    mlflow.end_run()
    return pipeline

In [ ]:
with mlflow.start_run(run_name="LightGBM_Feature_Engineering") as run:

    mlflow.log_param("time_features_added", ["Day", "Week", "Month", "Year", "Quarter", "DayOfYear", "WeekOfMonth"])
    mlflow.log_param("lag_features", [1, 2, 4, 8, 12])
    mlflow.log_param("rolling_windows", [4, 8, 12])
    mlflow.log_param("categorical_encoding", "LabelEncoder")
    mlflow.log_param("negative_sales_handling", "removed")
    mlflow.log_param("missing_value_strategy", "median_fill")

    print(f"Feature engineering parameters logged")
    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")

    mlflow.log_metric("processed_train_rows", train_df.shape[0])
    mlflow.log_metric("processed_train_cols", train_df.shape[1])
    mlflow.log_metric("processed_test_rows", test_df.shape[0])
    mlflow.log_metric("processed_test_cols", test_df.shape[1])

mlflow.end_run()

Feature engineering parameters logged
Training data shape: (420285, 5)
Test data shape: (115064, 4)


2025/07/15 11:09:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Feature_Engineering at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/4d5d986321ff4666b965dd0fa10317f1.
2025/07/15 11:09:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.


In [ ]:
print("STARTING LIGHTGBM EXPERIMENT")

X_train=train_df.drop('Weekly_Sales', axis=1)
y_train=train_df['Weekly_Sales']

print(f"Training data shape: {X_train.shape}")
print(f"Target shape: {y_train.shape}")

print("Running cross validation...")
cv_scores, avg_wmae=run_cross_validation(train_df)

print("Training final model...")
my_pipeline=train_model(train_df)


STARTING LIGHTGBM EXPERIMENT
Training data shape: (420285, 4)
Target shape: (420285,)

Running cross validation...
Working on fold 1/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	valid_0's l1: 15098.8
  MAE: 15098.83, RMSE: 23202.30, R2: 0.0001, WMAE: 15665.83
Working on fold 2/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's l1: 15135.9
  MAE: 15135.90, RMSE: 22604.59, R2: -0.0006, WMAE: 14980.71
Working on fold 3/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	valid_0's l1: 15008.9
  MAE: 15008.94, RMSE: 21887.28, R2: -0.0003, WMAE: 15055.80
Working on fold 4/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's l1: 15772.4
  MAE: 15772.36, RMSE: 24701.07, R2: -0.0010, WMAE: 16078.85
Working on fold 5/5
Training until validation scores don't improve fo

2025/07/15 11:20:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Cross_Validation at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/0f57ec90235243ec8a37a49c1535cd18.
2025/07/15 11:20:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.



CV results:
Average MAE: 15243.16
Average RMSE: 22892.41
Average R2: -0.0004
Average WMAE: 15409.37

Training final model...


2025/07/15 11:31:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpki9z_xny/model, flavor: lightgbm). Fall back to return ['lightgbm==4.5.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
Successfully registered model 'walmart_lightgbm_model'.
2025/07/15 11:32:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: walmart_lightgbm_model, version 1
Created version '1' of model 'walmart_lightgbm_model'.
2025/07/15 11:32:02 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/07/15 11:32:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Final_Training at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/94709e7abcc349fe9977c4134495c36b.
2025/07/15 11:32:02 INFO mlflow.tr

Model training metrics:
MAE: 1294.69, RMSE: 2402.67, R2: 0.9888


In [ ]:
with mlflow.start_run(run_name="LightGBM_Submission_Generation") as run:
    print("CREATING SUBMISSION FILE")

    test_predictions=my_pipeline.predict(test_df)
    test_predictions=np.maximum(test_predictions, 0)

    test_submission=test_df[['Store', 'Dept', 'Date']].copy()
    test_submission['Weekly_Sales']=test_predictions

    test_submission['Date_str']=pd.to_datetime(test_submission['Date']).dt.strftime('%Y-%m-%d')
    test_submission['Id']=(test_submission['Store'].astype(str)+'_' +
                           test_submission['Dept'].astype(str)+'_' +
                           test_submission['Date_str'])

    submission=test_submission[['Id', 'Weekly_Sales']].copy()

    submission_path=f"{FOLDERNAME}/submissions/lightgbm_submission.csv"
    submission.to_csv(submission_path, index=False)

    mlflow.log_artifact(submission_path)
    mlflow.log_metric("submission_rows", len(submission))
    mlflow.log_metric("prediction_mean", np.mean(test_predictions))
    mlflow.log_metric("prediction_std", np.std(test_predictions))
    mlflow.log_metric("prediction_min", np.min(test_predictions))
    mlflow.log_metric("prediction_max", np.max(test_predictions))

    print(f"Submission saved to: {submission_path}")
    print(f"Submission shape: {submission.shape}")
    print("First 10 rows of submission:")
    print(submission.head(10))

mlflow.end_run()


CREATING SUBMISSION FILE


2025/07/15 11:33:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_Submission_Generation at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1/runs/31bd114cd96d4e949fd254dc95a17cd6.
2025/07/15 11:33:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skara-21/ML_Final_Project.mlflow/#/experiments/1.


Submission saved to: /content/drive/MyDrive/ML_final_project/submissions/lightgbm_submission.csv
Submission shape: (115064, 2)

First 10 rows of submission:
               Id  Weekly_Sales
0  1_1_2012-11-02  33919.338495
1  1_1_2012-11-09  22639.544689
2  1_1_2012-11-16  20724.343998
3  1_1_2012-11-23  20847.508884
4  1_1_2012-11-30  25396.228025
5  1_1_2012-12-07  30506.103265
6  1_1_2012-12-14  43249.528919
7  1_1_2012-12-21  54777.411543
8  1_1_2012-12-28  29845.165822
9  1_1_2013-01-04  17231.132986


In [ ]:
model_path=f"{FOLDERNAME}/submissions/lightgbm_pipeline.pkl"
joblib.dump(my_pipeline, model_path)
print(f"Pipeline saved to: {model_path}")

ეს იმიტომ რომ ლინკებს არ მიგდებდა MLFlow-სთვის

In [ ]:
print("MLflow Tracking Information:")
print(f"Experiment Name: LightGBM_Training")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print("To view results, you can access MLflow UI or check the experiment in your MLflow server.")

client = MlflowClient()
experiment = client.get_experiment_by_name("LightGBM_Training")
if experiment:
    runs = client.search_runs(experiment_ids=[experiment.experiment_id], max_results=10)
    print(f"Recent runs in this experiment:")
    for run in runs:
        print(f"- {run.info.run_name} (Status: {run.info.status})")


MLflow Tracking Information:
Experiment Name: LightGBM_Training
Tracking URI: https://dagshub.com/skara-21/ML_Final_Project.mlflow

To view results, you can access MLflow UI or check the experiment in your MLflow server.

Recent runs in this experiment:
- LightGBM_Submission_Generation (Status: FINISHED)
- LightGBM_Final_Training (Status: FINISHED)
- LightGBM_Cross_Validation (Status: FINISHED)
- LightGBM_Feature_Engineering (Status: FINISHED)
- LightGBM_Missing_Values (Status: FINISHED)
- LightGBM_Cleaning (Status: FINISHED)
- LightGBM_Data_Loading (Status: FINISHED)
